In [31]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [32]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [33]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_CODE', 'des_col': 'dtr_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_NAME', 'des_col': 'dtr_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AREA_ID', 'des_col': 'area_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBAREA_ID', 'des_col': 'subarea_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEEDER_ID', 'des_col': 'feeder_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_LOC', 'des_col': 'dtr_loc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MAX_CAPACITY', 'des_col': 'max_capacity', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LOAD_UNIT', 'des_col': 'load_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE', 'des_col': 'phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRIMARY_VOLTAGE', 'des_col': 'primary_voltage', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SECONDARY_VOLTAGE', 'des_col': 'secondary_voltage', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRIMARY_FUSE_SIZE', 'des_col': 'primary_fuse_size', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SECONDARY_FUSE_SIZE', 'des_col': 'secondary_fuse_size', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DTR_MAKE', 'des_col': 'dtr_make', 'src_dtype': 'String', 'des_dtype': 'String'},
    #{'src_col': 'INSTALLATTION_DT', 'des_col': 'installattion_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'DTR_SL_NO', 'des_col': 'dtr_sl_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MF', 'des_col': 'mf', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'INITIAL_READING', 'des_col': 'initial_reading', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'INITIAL_READING_DATE', 'des_col': 'initial_reading_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'OWNED_BY', 'des_col': 'owned_by', 'src_dtype': 'DECFLOAT', 'des_dtype': 'numeric'},
    {'src_col': 'GISUID', 'des_col': 'gisuid', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POS_NUM', 'des_col': 'pos_num', 'src_dtype': 'DECIMAL', 'des_dtype': 'bigint'},
    #{'src_col': 'GUARANTEE_DATE', 'des_col': 'guarantee_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'DTR_AVAILABLE', 'des_col': 'dtr_available', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANSFORMER_LAT', 'des_col': 'transformer_lat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANSFORMER_LONG', 'des_col': 'transformer_long', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POSITION', 'des_col': 'position', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TYPE', 'des_col': 'type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_OF_CONNECTIONS', 'des_col': 'no_of_connections', 'src_dtype': 'DECIMAL', 'des_dtype': 'integer'},
    {'src_col': 'CONNECTED_LOAD', 'des_col': 'connected_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PROVIENCE_CODE', 'des_col': 'provience_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PR_TOWN', 'des_col': 'pr_town', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBSTATION_CODE', 'des_col': 'substation_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GISUCODE', 'des_col': 'gisucode', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_REL', 'des_col': 'dtr_rel', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_MAP_CODE', 'des_col': 'dtr_map_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EXISTANCE_ENERGY_MTR', 'des_col': 'existance_energy_mtr', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ENERGY_MTR_SIDE', 'des_col': 'energy_mtr_side', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBSTATION_TYPE', 'des_col': 'substation_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_RDR', 'des_col': 'mtr_rdr', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEEDER_VOLT_LEVEL', 'des_col': 'feeder_volt_level', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MEDIUM_VOLTAGE_FEEDER', 'des_col': 'medium_voltage_feeder', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AIN', 'des_col': 'ain', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_CAT', 'des_col': 'dtr_cat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_WRK_LOC', 'des_col': 'dtr_wrk_loc', 'src_dtype': 'String', 'des_dtype': 'String'}

]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.DTR_M', 'public', 'dtr_m', cols_map)
print("Total Migrated Records (dtr_m): " + str(migrated_row_count))


Total Migrated Records (dtr_m): Total Src Records: 11201 Total migrated Records to dest: 11201
